In [2]:
pip install --upgrade google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [3]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [4]:
api_key = ''
channel_id = "UCmFj0ZKQpGWmxW3pcz2U4QQ"   
youtube= build('youtube','v3',developerKey=api_key) #created a youtube service 
channel_ids = ["UCnz-ZXXER4jOvuED5trXfEA","UCjWY5hREA6FFYrthD0rZNIw","UCBJycsmduvYEL83R_U4JriQ","UCb1GdqUqArXMQ3RS86lqqOw",
               "UCiT9RITQ9PW6BhXK0y2jaeg"]

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
# defining function to get channel statistics , it will return channel name, statistics i.e., 
def get_channel_stats(youtube,channel_id):
    all_data=[]
    request = youtube.channels().list(
            part="snippet,contentDetails,statistics",
            id=",".join(channel_ids)
        )
    response= request.execute()
    for i in range(len(response['items'])):
        data = dict(Channel_name= response['items'][i]['snippet']['title'],
                    Playlist_Id= response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                    Subscribers=response['items'][i]['statistics']['subscriberCount'],
                    Views=response['items'][i]['statistics']['viewCount'],
                    Total_videos=response['items'][i]['statistics']['videoCount'])
        all_data.append(data)
    return all_data

In [ ]:
channel_statistics=get_channel_stats(youtube,channel_ids)

In [ ]:
channel_statistics

[{'Channel_name': 'Ken Jee',
  'Playlist_Id': 'UUiT9RITQ9PW6BhXK0y2jaeg',
  'Subscribers': '249000',
  'Views': '8419999',
  'Total_videos': '282'},
 {'Channel_name': 'iNeuron Intelligence',
  'Playlist_Id': 'UUb1GdqUqArXMQ3RS86lqqOw',
  'Subscribers': '62800',
  'Views': '364878',
  'Total_videos': '99'},
 {'Channel_name': 'Krish Naik Hindi',
  'Playlist_Id': 'UUjWY5hREA6FFYrthD0rZNIw',
  'Subscribers': '68400',
  'Views': '2786882',
  'Total_videos': '397'},
 {'Channel_name': 'techTFQ',
  'Playlist_Id': 'UUnz-ZXXER4jOvuED5trXfEA',
  'Subscribers': '229000',
  'Views': '11582786',
  'Total_videos': '94'},
 {'Channel_name': 'Marques Brownlee',
  'Playlist_Id': 'UUBJycsmduvYEL83R_U4JriQ',
  'Subscribers': '17400000',
  'Views': '3693432826',
  'Total_videos': '1576'}]

In [ ]:
channel_data= pd.DataFrame(channel_statistics)
channel_data

,Channel_name,Playlist_Id,Subscribers,Views,Total_videos
0,Ken Jee,UUiT9RITQ9PW6BhXK0y2jaeg,249000,8419999,282
1,iNeuron Intelligence,UUb1GdqUqArXMQ3RS86lqqOw,62800,364878,99
2,Krish Naik Hindi,UUjWY5hREA6FFYrthD0rZNIw,68400,2786882,397
3,techTFQ,UUnz-ZXXER4jOvuED5trXfEA,229000,11582786,94
4,Marques Brownlee,UUBJycsmduvYEL83R_U4JriQ,17400000,3693432826,1576


In [ ]:
channel_data.dtypes

In [ ]:
channel_data['Subscribers']=pd.to_numeric(channel_data['Subscribers'])
channel_data['Views']=pd.to_numeric(channel_data['Views'])
channel_data['Total_videos']=pd.to_numeric(channel_data['Total_videos'])
channel_data.dtypes

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.barplot(x='Channel_name',y='Subscribers',data=channel_data)

In [ ]:
sns.barplot(x='Channel_name',y='Views',data=channel_data)


In [ ]:
sns.barplot(x='Channel_name',y='Total_videos',data=channel_data)


In [ ]:
#so we have extracted the channel details for a few channels and then visualized them,
#Now we will try to scrap, analyze, visualize video details for a particular channel.

In [ ]:
playlist_id= channel_data.loc[channel_data['Channel_name']=='Ken Jee','Playlist_Id'].iloc[0]
playlist_id

In [ ]:
## defining function to get video ids
'''
def get_video_ids(_youtube,playlist_id):
    
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50
    )
    response= request.execute()

    video_ids=[]
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
    return video_ids
    '''

In [ ]:
#get_video_ids(youtube,playlist_id)

In [ ]:
request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50
)
response= request.execute()
response

# it can be seen from the bottom of output , the page infor shows that we are getting 5 results
#  as output while there are 282 more records, so to get all records 
# we have to set parameter maxResults=50 , so if all records are under 50 then all will be shown ,
#  but if higher than 50 then there will be "nextPageToken" shown in output , it will be needed to fetch all the 282 records.

#By putting all the output in any jsonformatter  , we can see it in a structured manner and
# there in that format of dictionary , we can see nextPageToken 

In [ ]:
## defining function to get video ids
def get_video_ids(_youtube,playlist_id):
    
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50
    )
    response= request.execute()

    video_ids=[]
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
    _nextPageToken= response.get('nextPageToken')
    morePages=True
    while morePages:
        if _nextPageToken is None:
            morePages= False
        else:
            request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=_nextPageToken)
            response= request.execute()

            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
                
            _nextPageToken= response.get('nextPageToken')

    return video_ids

In [ ]:
video_ids=get_video_ids(youtube,playlist_id)
video_ids


In [ ]:
##defining function to get details of the viddos using videoids
def get_video_details(youtube,video_ids):
    all_video_stats=[]
    for i in range(0,len(video_ids),50):
        request= youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids[i:i+50]))
        response = request.execute()

        for video in response['items']:
            video_stats=dict(Title=video['snippet']['title'],
                            Published_date= video['snippet']['publishedAt'],
                            Views= video['statistics']['viewCount'],
                            Likes= video['statistics']['likeCount'],
                            Favourites= video['statistics']['favoriteCount'],
                            Comments= video['statistics']['commentCount'])

            all_video_stats.append(video_stats)
    return all_video_stats


In [ ]:
get_video_details(youtube , video_ids )

In [ ]:
def get_video_details1(youtube,video_ids):
     request=youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids[:50]))
     response = request.execute()
     return response

In [ ]:
Video_Details=get_video_details(youtube, video_ids)
Video_Details

In [ ]:
video_data= pd.DataFrame(Video_Details)
video_data

In [ ]:
video_data['Published_date']= pd.to_datetime(video_data['Published_date']).dt.date
video_data['Views']=pd.to_numeric(video_data['Views'])
video_data['Likes']=pd.to_numeric(video_data['Likes'])
video_data['Comments']=pd.to_numeric(video_data['Comments'])
video_data['Favourites']=pd.to_numeric(video_data['Favourites'])

In [ ]:
video_data

In [ ]:
top_10_videos= video_data.sort_values(by='Views',ascending=False).head(10)
top_10_videos


In [ ]:
ax1= sns.barplot(x='Views',y='Title', data= top_10_videos)

In [ ]:
#on an average how much video is he posting

In [ ]:
video_data['Month']= pd.to_datetime(video_data['Published_date']).dt.strftime('%b')
video_data

In [ ]:
videos_per_month= video_data.groupby('Month', as_index=False).size()
videos_per_month

In [ ]:
sort_order=['Jan','Feb','Mar','Apr','May','Jun',
            'Jul','Aug','Sep','Oct','Nov','Dec']

In [ ]:
videos_per_month.index= pd.CategoricalIndex(videos_per_month['Month'],categories=sort_order,ordered=True)

In [ ]:
videos_per_month=videos_per_month.sort_index()

In [ ]:
ax2= sns.barplot(x='Month',y='size',data=videos_per_month)

In [ ]:
video_data.to_csv('videoDetails_ken_jee.csv')